In [ ]:
%reload_ext autoreload
%autoreload 2

## nbmd:nbmake_timeout_300

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [105]:
from typing import List, Dict, Tuple
from pathlib import Path
import nbformat
import tempfile
from contextlib import contextmanager
from icecream import ic
import json
from nbmetadata.app import main as nbmd

@contextmanager
def create_notebook(cells: List[nbformat.notebooknode.NotebookNode] = [], metadata: Dict = {}) -> Path:
    with tempfile.NamedTemporaryFile(suffix='.ipynb', mode="w") as tf:
        nb = nbformat.v4.new_notebook()
        nb.cells = cells
        nb.metadata = metadata
        nbformat.write(nb, tf)

        tf.flush()
        
        # Both nb and the filehandle are not really valid once we run nbmd(). We
        # also need to be careful about flush(); instead just return the path
        yield Path(tf.name)

def get_notebook_json(f: Path):
    return json.load(f.open())

In [ ]:
## nbmd:nbmake_skip
raise RuntimeError("This cell should be skipped")

In [117]:
# 1. given a comment -> metadata is added
with create_notebook(cells=[nbformat.v4.new_code_cell("## nbmd:nbmake_skip\nfoo")]) as nb:
    nbmd([nb])
    nb_json = get_notebook_json(nb)
    assert nb_json['cells'][0]['metadata']['tags'] == ['skip-execution']
    assert nb_json['cells'][0]['metadata']['_nbmd_keys'] == ['tags']
    
    # 2. remove the comment -> metadata goes away
    nbnode = nbformat.read(nb, as_version=nbformat.NO_CONVERT)
    nbnode.cells[0].source = ['foo']
    nbformat.write(nbnode, nb)
    nbmd([nb])
    nb_json = get_notebook_json(nb)
    assert 'tags' not in nb_json['cells'][0]['metadata']
    assert '_nbmd_keys' not in nb_json['cells'][0]['metadata']
    

In [118]:
# 3. no comment or non-magic comment -> no impact
with create_notebook(cells=[nbformat.v4.new_code_cell("foo")]) as nb:
    nbmd([nb])
    nb_json = get_notebook_json(nb)
    assert 'tags' not in nb_json['cells'][0]['metadata']
    assert '_nbmd_keys' not in nb_json['cells'][0]['metadata']


In [120]:
## nbmd:nbmake_raises
# 4. magic comment with typo -> error
with create_notebook(cells=[nbformat.v4.new_code_cell("## nbmd:foo\nfoo")]) as nb:
    nbmd([nb])


KeyError: 'foo'

In [121]:
# 5. top-level comment -> works
with create_notebook(cells=[nbformat.v4.new_code_cell("## nbmd:nbmake_timeout_300\nfoo")]) as nb:
    nbmd([nb])
    nb_json = get_notebook_json(nb)
    assert nb_json['metadata']['execution'] == {'timeout': 300}
    assert nb_json['metadata']['_nbmd_keys'] == ['execution']

